In [23]:
import time
import numpy as np
import pandas as pd
import xgboost as xgb
from datetime import datetime
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from bayes_opt import BayesianOptimization
from sklearn.metrics import log_loss
seed = 1234

In [3]:
train_X = pd.read_pickle('../input/' + 'train_X_2017-02-16-20-51.pkl')
test_X = pd.read_pickle('../input/' + 'test_X_2017-02-16-20-51.pkl')
test_listing = pd.read_pickle('../input/' + 'listing_id.pkl')

train_y = pd.read_pickle('../input/' + 'y_2017-02-16-20-51.pkl') 
print train_X.shape, test_X.shape

(49352, 229) (74659, 229)


In [3]:
# SEED = 777
# NFOLDS = 5

# params = {
#     'eta':.01,
#     'colsample_bytree':.8,
#     'subsample':.8,
#     'seed':0,
#     'nthread':8,
#     'objective':'multi:softprob',
#     'eval_metric':'mlogloss',
#     'num_class':3,
#     'silent':1
# }

# dtrain = xgb.DMatrix(data=train_X, label=train_y)
# dtest = xgb.DMatrix(data=test_X)


# bst = xgb.cv(params, dtrain, 10000, NFOLDS, early_stopping_rounds=50, verbose_eval=25)

# best_rounds = np.argmin(bst['test-mlogloss-mean'])



[0]	train-mlogloss:1.09181+1.54583e-05	test-mlogloss:1.09193+1.43889e-05
[25]	train-mlogloss:0.952515+0.000228609	test-mlogloss:0.955351+0.00052189
[50]	train-mlogloss:0.855399+0.000530515	test-mlogloss:0.860855+0.00083777
[75]	train-mlogloss:0.78536+0.000644246	test-mlogloss:0.793402+0.0010361
[100]	train-mlogloss:0.733626+0.00088164	test-mlogloss:0.744086+0.00134885
[125]	train-mlogloss:0.694538+0.000931174	test-mlogloss:0.707257+0.00148517
[150]	train-mlogloss:0.664556+0.000966226	test-mlogloss:0.679356+0.0018101
[175]	train-mlogloss:0.640918+0.00106951	test-mlogloss:0.657685+0.00197618
[200]	train-mlogloss:0.622062+0.00115715	test-mlogloss:0.640714+0.00218976
[225]	train-mlogloss:0.606891+0.00118513	test-mlogloss:0.627324+0.00248528
[250]	train-mlogloss:0.594301+0.00127697	test-mlogloss:0.616446+0.00260129
[275]	train-mlogloss:0.583457+0.00122491	test-mlogloss:0.607283+0.00282843
[300]	train-mlogloss:0.574255+0.00126443	test-mlogloss:0.599781+0.00297424
[325]	train-mlogloss:0.56637

In [10]:
X_train, X_val, y_train, y_val = train_test_split(train_X, train_y, train_size=.80, random_state=1234)

In [24]:
learning_rate = 0.1
for x in [5,6,7,8,9,10]:
    rgr = xgb.XGBClassifier(
        objective='multi:softprob',
        seed = 1234, # use a fixed seed during tuning so we can reproduce the results
        learning_rate = learning_rate,
        n_estimators = 10000,
        max_depth= x,
        nthread = -1,
        silent = False
    )
    rgr.fit(
        X_train,y_train,
        eval_set=[(X_val,y_val)],
        eval_metric='mlogloss',
        early_stopping_rounds=20,
        verbose=False
    )

    # print 'max_depth:\t{}\n'.format(rgr.get_xgb_params()['max_depth'])
    # print 'score    :\t{}\n'.format(rgr.best_score)

    print rgr.get_xgb_params()['max_depth'], ' ', rgr.best_score

5   0.535599
6   0.537981
7   0.539522
8   0.541843
9   0.54299
10   0.548744


In [25]:
max_depth = 5

In [28]:
for x in [130,140,150,160,170,180,190]:
    rgr = xgb.XGBClassifier(
        objective='multi:softprob',
        seed = 1234, # use a fixed seed during tuning so we can reproduce the results
        learning_rate = learning_rate,
        n_estimators = 10000,
        max_depth= max_depth,
        nthread = -1,
        silent = False,
        min_child_weight = x
    )
    rgr.fit(
        X_train,y_train,
        eval_set=[(X_val,y_val)],
        eval_metric='mlogloss',
        early_stopping_rounds=20,
        verbose=False
    )

    # print 'max_depth:\t{}\n'.format(rgr.get_xgb_params()['max_depth'])
    # print 'score    :\t{}\n'.format(rgr.best_score)

    print rgr.get_xgb_params()['min_child_weight'], ' ', rgr.best_score

130   0.538513
140   0.537914
150   0.539292
160   0.539588
170   0.540907
180   0.541307
190   0.542417


In [29]:
min_child_weight = 10
# 1   0.535599
# 5   0.53421
# 10   0.534681
# 20   0.534445
# 30   0.536634
# 40   0.537307
# 50   0.536874
# 60   0.538
# 70   0.537298
# 80   0.538018
# 90   0.537785
# 100   0.536762
# 110   0.538405
# 120   0.536963
# 130   0.538513
# 140   0.537914
# 150   0.539292
# 160   0.539588
# 170   0.540907
# 180   0.541307
# 190   0.542417


In [30]:
for x in [0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]:
    rgr = xgb.XGBClassifier(
        objective='multi:softprob',
        seed = 1234, # use a fixed seed during tuning so we can reproduce the results
        learning_rate = learning_rate,
        n_estimators = 10000,
        max_depth= max_depth,
        nthread = -1,
        silent = False,
        min_child_weight = min_child_weight,
        colsample_bytree = x
    )
    rgr.fit(
        X_train,y_train,
        eval_set=[(X_val,y_val)],
        eval_metric='mlogloss',
        early_stopping_rounds=20,
        verbose=False
    )

    # print 'max_depth:\t{}\n'.format(rgr.get_xgb_params()['max_depth'])
    # print 'score    :\t{}\n'.format(rgr.best_score)

    print rgr.get_xgb_params()['colsample_bytree'], ' ', rgr.best_score

0.3   0.533581
0.4   0.536133
0.5   0.533623
0.6   0.533882
0.7   0.534225
0.8   0.532032
0.9   0.533586
1.0   0.534681


In [31]:
colsample_bytree = 0.8

In [34]:
for x in [0.5,0.6,0.7,0.8,0.9,1.0]:
    rgr = xgb.XGBClassifier(
        objective='multi:softprob',
        seed = 1234, # use a fixed seed during tuning so we can reproduce the results
        learning_rate = learning_rate,
        n_estimators = 10000,
        max_depth= max_depth,
        nthread = -1,
        silent = False,
        min_child_weight = min_child_weight,
        colsample_bytree = colsample_bytree,
        subsample = x
    )
    rgr.fit(
        X_train,y_train,
        eval_set=[(X_val,y_val)],
        eval_metric='mlogloss',
        early_stopping_rounds=20,
        verbose=False
    )

    # print 'max_depth:\t{}\n'.format(rgr.get_xgb_params()['max_depth'])
    # print 'score    :\t{}\n'.format(rgr.best_score)

    print rgr.get_xgb_params()['subsample'], ' ', rgr.best_score

0.5   0.534732
0.6   0.535824
0.7   0.53323
0.8   0.531295
0.9   0.53185
1.0   0.532032


In [35]:
subsample = 0.8

In [36]:
for x in [0, 0.3, 0.6, 0.9, 1.2, 1.5, 1.8, 2.1, 2.4, 2.7, 3.0]:
    rgr = xgb.XGBClassifier(
        objective='multi:softprob',
        seed = 1234, # use a fixed seed during tuning so we can reproduce the results
        learning_rate = learning_rate,
        n_estimators = 10000,
        max_depth= max_depth,
        nthread = -1,
        silent = False,
        min_child_weight = min_child_weight,
        colsample_bytree = colsample_bytree,
        subsample = subsample,
        gamma = x
    )
    rgr.fit(
        X_train,y_train,
        eval_set=[(X_val,y_val)],
        eval_metric='mlogloss',
        early_stopping_rounds=20,
        verbose=False
    )

    # print 'max_depth:\t{}\n'.format(rgr.get_xgb_params()['max_depth'])
    # print 'score    :\t{}\n'.format(rgr.best_score)

    print rgr.get_xgb_params()['gamma'], ' ', rgr.best_score

0   0.531295
0.3   0.532525
0.6   0.533081
0.9   0.533356
1.2   0.532935
1.5   0.533338
1.8   0.533093
2.1   0.532699
2.4   0.533399
2.7   0.531466
3.0   0.532402


In [37]:
gamma = 1.2

In [58]:
xgtrain = xgb.DMatrix(train_X, label=train_y) 

def xgb_evaluate(min_child_weight, colsample_bytree, max_depth, subsample, gamma):
    params = dict()
    params['objective']='multi:softprob'
    params['eval_metric']='mlogloss',
    params['num_class']=3
    params['silent']=1
    params['eta'] = 0.1
    params['verbose_eval'] = True
    params['min_child_weight'] = int(min_child_weight)
    params['colsample_bytree'] = max(min(colsample_bytree, 1), 0)
    params['max_depth'] = int(max_depth)
    params['subsample'] = max(min(subsample, 1), 0)
    params['gamma'] = max(gamma, 0)
    
    cv_result = xgb.cv(
        params, xgtrain, 
        num_boost_round=10000, nfold=5,
        metrics = 'mlogloss',
        seed=seed,callbacks=[xgb.callback.early_stop(50)]
    )
    
    return -cv_result['test-mlogloss-mean'].values[-1]


xgb_BO = BayesianOptimization(
    xgb_evaluate, 
    {
        'max_depth': (3,7),
        'min_child_weight': (1,50),
        'colsample_bytree': (0.7,1),
        'subsample': (0.7,1),
        'gamma': (0,3)
    }
)

xgb_BO.maximize(init_points=10, n_iter=40)

Initialization
---------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   colsample_bytree |     gamma |   max_depth |   min_child_weight |   subsample | 
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 50 rounds.


KeyboardInterrupt: 

In [52]:
xgb_bo_scores = pd.DataFrame([[s[0]['max_depth'],
                               s[0]['min_child_weight'],
                               s[0]['colsample_bytree'],
                               s[0]['subsample'],
                               s[0]['gamma'],
                               s[1]] for s in zip(xgb_BO.res['all']['params'],xgb_BO.res['all']['values'])],
                            columns = ['max_depth',
                                       'min_child_weight',
                                       'colsample_bytree',
                                       'subsample',
                                       'gamma',
                                       'score'])
xgb_bo_scores=xgb_bo_scores.sort_values('score',ascending=False)
xgb_bo_scores.head(10)

,max_depth,min_child_weight,colsample_bytree,subsample,gamma,score
22,5.884599,12.897203,0.803018,0.960795,2.690196,-0.530282
12,4.126611,2.067154,0.828660,0.821156,2.751725,-0.530455
3,4.126611,2.067154,0.828660,0.821156,2.751725,-0.530455
14,4.126611,2.067160,0.828660,0.821156,2.751725,-0.530455
13,4.126611,2.067154,0.828660,0.821156,2.751725,-0.530455
16,5.897646,12.958748,0.736776,0.947351,2.677209,-0.530621
10,5.897647,12.958754,0.736769,0.947350,2.677208,-0.530917
18,4.698249,7.818157,0.838006,0.930783,2.668471,-0.530937
0,6.092796,25.284920,0.932358,0.949351,2.942120,-0.530973
1,5.441614,1.988824,0.976038,0.791367,2.169951,-0.531241


In [61]:
def xgb_blend(params, train_x, train_y, test_x, fold, early_stopping_rounds=0):
    N_params = len(params)
    print ("Blend %d estimators for %d folds" % (N_params, fold))
    skf = KFold(n_splits=fold,random_state=seed)
    N_class = len(set(train_y))
        
    train_blend_x = np.zeros((train_x.shape[0], N_class*N_params))
    test_blend_x = np.zeros((test_x.shape[0], N_class*N_params))
    scores = np.zeros ((fold,N_params))
    best_rounds = np.zeros ((N_params))
    
    for j, param in enumerate(params):
        param['objective']='multi:softprob'
        param['eval_metric']='mlogloss',
        param['num_class']=3
        param['silent']= False
        param['eta'] = 0.03
#         param['verbose_eval'] = 10  
        print ("Model %d:" %(j+1))
        
        xgtrain = xgb.DMatrix(train_X, label=train_y)
        cv_result = xgb.cv(param, xgtrain,
                           num_boost_round=10000, nfold=fold,
                           metrics = 'mlogloss',
                           seed=seed,callbacks=[xgb.callback.early_stop(early_stopping_rounds)])    
        best_round = cv_result.shape[0] - 1
        print 'best_round',best_round
        best_rounds[j]=best_round
        
        param.pop('eval_metric')
        all_round = best_round / (1 - 1. / fold)
        est_test_blend = xgb.train(param, xgtrain,num_boost_round=int(all_round))

        test_blend_x[:,(j*N_class):(j+1)*N_class] = est_test_blend.predict(xgb.DMatrix(test_x))

        for i, (train_index, val_index) in enumerate(skf.split(train_x)):
            print ("Model %d fold %d" %(j+1,i+1))
            fold_start = time.time() 
            train_x_fold = train_x[train_index]
            train_y_fold = train_y[train_index]
            val_x_fold = train_x[val_index]
            val_y_fold = train_y[val_index]
            
            xgtrain_fold = xgb.DMatrix(train_x_fold, label=train_y_fold)
            est_train_blend = xgb.train(param, xgtrain_fold,num_boost_round=best_round)
            
            val_y_predict_fold = est_train_blend.predict(xgb.DMatrix(val_x_fold))
            score = log_loss(val_y_fold, val_y_predict_fold)
            print ("Score: ", score)
            scores[i,j]=score            
            
            train_blend_x[val_index, (j*N_class):(j+1)*N_class] = val_y_predict_fold
            print ("Model %d fold %d fitting finished in %0.3fs" % (j+1,i+1, time.time() - fold_start))            

        print ("Score for model %d is %f" % (j+1,np.mean(scores[:,j])))
    print ("Score for blended models is %f" % (np.mean(scores)))
    return (train_blend_x, test_blend_x, scores,best_rounds)


In [62]:

xgb_params = [{'max_depth':5,
               'min_child_weight':12,
               'colsample_bytree':0.803018,
               'subsample':0.960795,
               'gamma':2.690196},
#               score -0.530282              

              {'max_depth':4,
               'min_child_weight':2,
               'colsample_bytree':0.828660,
               'subsample':0.821156,
               'gamma':2.751725},
#               score -0.530455
          
              {'max_depth':5,
               'min_child_weight':12,
               'colsample_bytree':0.736776,
               'subsample':0.947351,
               'gamma':2.677209},
#               score -0.530621    
              
              {'max_depth':5,
               'min_child_weight':12,
               'colsample_bytree':0.736769,
               'subsample':0.947350,
               'gamma':2.677208},
#               score -0.530917                 

              {'max_depth':4,
               'min_child_weight':7,
               'colsample_bytree':0.838006,
               'subsample':0.930783,
               'gamma':2.668471}
#               score -0.530937
             ]

(train_blend_x_xgb,
 test_blend_x_xgb,
 blend_scores_xgb,
 best_rounds_xgb) = xgb_blend(xgb_params,
                              train_X,train_y,
                              test_X,
                              5,
                              300)

# print (np.mean(blend_scores_xgb_le,axis=0))
# print (np.mean(best_rounds_xgb_le,axis=0))

Blend 5 estimators for 5 folds
Model 1:
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 300 rounds.
Stopping. Best iteration:
[2794]	train-mlogloss:0.402639+0.00175193	test-mlogloss:0.52947+0.0068331

best_round 2794
Model 1 fold 1
('Score: ', 0.53690951454152624)
Model 1 fold 1 fitting finished in 388.677s
Model 1 fold 2
('Score: ', 0.52401377183494546)
Model 1 fold 2 fitting finished in 368.404s
Model 1 fold 3
('Score: ', 0.52100373678403)
Model 1 fold 3 fitting finished in 370.817s
Model 1 fold 4
('Score: ', 0.53049319689493357)
Model 1 fold 4 fitting finished in 381.897s
Model 1 fold 5
('Score: ', 0.53938186191760162)
Model 1 fold 5 fitting finished in 369.014s
Score for model 1 is 0.530360
Model 2:
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 300 rounds.
Stopping. Best iteration:
[4134]	train-mlog

In [63]:
best_rounds_xgb

array([ 2794.,  4134.,  2798.,  3053.,  4475.])

In [65]:
now = datetime.now()

name_train_blend = '../output/train_blend_xgb_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'
name_test_blend = '../output/test_blend_xgb_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'



print (np.mean(blend_scores_xgb,axis=0))
print (np.mean(best_rounds_xgb,axis=0))
np.savetxt(name_train_blend,train_blend_x_xgb, delimiter=",")
np.savetxt(name_test_blend,test_blend_x_xgb, delimiter=",")

[ 0.53036042  0.52949224  0.53015283  0.53017668  0.53011856]
3450.8


In [71]:
test_blend_x_xgb[:,:3]


array([[ 0.13749345,  0.49183246,  0.37067413],
       [ 0.0594456 ,  0.09174278,  0.84881157],
       [ 0.02579494,  0.27639574,  0.69780934],
       ..., 
       [ 0.13193671,  0.36944345,  0.49861979],
       [ 0.40039733,  0.48122504,  0.11837757],
       [ 0.02176894,  0.17235944,  0.80587167]])

In [72]:
# now = datetime.now()
sub_name = '../output/sub_XGB_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'

out_df = pd.DataFrame(test_blend_x_xgb[:,:3])
out_df.columns = ["high", "medium", "low"]
out_df["listing_id"] = test_listing
out_df.to_csv(sub_name, index=False)


# ypreds.columns = cols

# df = pd.read_json(open("../input/test.json", "r"))
# ypreds['listing_id'] = df["listing_id"]

# ypreds.to_csv('my_preds.csv', index=None)

In [1]:
test_blend_x_xgb

NameError: name 'test_blend_x_xgb' is not defined